In [56]:
from collections import Counter
from operator import itemgetter
import pandas as pd
import numpy as np
import sys
!pip install indic-nlp-library
from indicnlp import common
INDIC_NLP_LIB_HOME = r"indic_nlp_library"
INDIC_NLP_RESOURCES = r"indic_nlp_resources"
sys.path.append(r'{}\src'.format(INDIC_NLP_LIB_HOME))
common.set_resources_path(INDIC_NLP_RESOURCES)

In [57]:
import pickle
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
def get_dict_from_pickle(filename):
  with open("drive/My Drive/"+filename, "rb") as temp:
    temp = pickle.load(temp)
    return temp

In [59]:
def remove_noise(word_gram, noise):
  temp = {}
  for key,val in word_gram.items():
    if(key not in noise):
      temp[key] = val
  return temp

In [60]:
def get_top_n(word_gram, n):
  word_with_freq = Counter(word_gram)
  word_with_freq = word_with_freq.most_common(100)
  # word_with_freq = dict(sorted(word_with_freq.items(), key = itemgetter(1), reverse = True)[:n])
  # df = pd.DataFrame()
  # df.columns = ['word','frequency']
  # df.word = np.array(word_with_freq.keys())
  # df.frequency = np.array([val for key,val in word_with_freq.keys()])
  df = pd.DataFrame.from_dict(word_with_freq)
  df.columns = ['word','frequency']
  df.sort_values('frequency',ascending=False,inplace=True)
  return df


## 3(b) Part 1: Unigram words

In [61]:
word_ug = get_dict_from_pickle('UNIGRAM')
## Removing noise from dictionary
noise = ['।',',','.','-','"',"'",'(',')',':']
word_ug = remove_noise(word_ug, noise)
top100_3b1 = get_top_n(word_ug,100)
top100_3b1

,word,frequency
0,के,66750756
1,में,50464902
2,है,44822080
3,की,41031653
4,को,30698906
...,...,...
95,लोग,1703804
96,वे,1697434
97,बहुत,1689441
98,रूप,1668308


## 3(b) Part 2: Bigrams

In [62]:
word_bg = get_dict_from_pickle('BIGRAM')
noise = []
noise = ['।',',','.']
word_bg = remove_noise(word_bg, noise)
top100_3b2 = get_top_n(word_bg,100)
top100_3b2

,word,frequency
0,है ।,19792620
1,के लिए,9123415
2,हैं ।,7420130
3,. .,7132228
4,है .,5635527
...,...,...
95,किया था,630478
96,सरकार ने,604986
97,के तहत,603831
98,जा सकता,602450


## 3(b) Part 3: Trigrams

In [63]:
word_tg = get_dict_from_pickle('TRIGRAM')
noise = ['।',',','.','-','(',')']
word_tg = remove_noise(word_tg, noise)
top100_3b3 = get_top_n(word_tg,100)
top100_3b3

,word,frequency
0,. . .,3859257
1,ने कहा कि,1509684
2,गया है ।,1274628
3,रहा है ।,1241743
4,करने के लिए,1094800
...,...,...
95,हो रही है,219434
96,किया जा सकता,218800
97,"रहा है ,",217641
98,लिया है ।,216259


## 3(a)

In [64]:
def need_halanta(word, index, vyanjan):
  ans = (word[index] in vyanjan) and (word[index-1] in vyanjan)
  return ans

In [65]:
def process_diacritic(word):
    vyanjan=['क','ख','ग','घ','ड़','च','छ','ज','झ','ञ','ट','ठ','ड','ढ','ण', 'त','थ','द','ध','न','प','फ','ब','भ','म','य','र','ल','व', 'श','ष','स','ह']

    wordlist=[]
    wordlist.append(word[0])
    ctr = 0
    for i in range(1, len(word)):   
        if (need_halanta(word, i, vyanjan)):
            wordlist.append('अ')
        wordlist.append(word[i])
        ctr += 1
    if word[-1] in vyanjan:
        wordlist.append('अ')
    wordlist2 = ([s for s in wordlist if s != '्'])
    ctr+=1
    # print(ctr)
    word = ''.join(wordlist2)
    
    return(word)

In [66]:
def remove_noise2(word_gram):
  noise = ['>्','0्', '1्', '"्', 'ळ्', '2्', '–्', 'ः्', '3्', '5्', '4्', '््', 
           '%्', '—्', '8्', '6्', '7्', '9्', 'ॅ्', 'a्', '>्', 'e्', '#्', 'i्',
           'r्', 't्', '»्', 'o्', 'n्', 'd्', '०्', 's्', 'h्', 'l्', 'c्', 'm्',
           '\u200c्', '\u200b्', 'ï्', 'A्', 'p्', '•्', 'b्', 'G्', 'B्', '&्', 'u्',
           '_्', '@्', 'M्', 'о्', 'f्', '·्', '$्', 'S्', 'g्', 'I्', 'а्', 'е्', 'P्',
           'и्', 'R्', 'y्', 'k्', 'w्', 'T्', '�्', 'a','b','c','d','e','f','g','h','i','j',
 'k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','0','1','2','3','4','5','6','7','8','9',
 'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z',
 '!','"','\\','#','\\','$','%','\\','&',"'",'\\','(','\\',')','\\','*','\\','+',',','\\','-','\\','.','/',
 ':',';','<','=','>','\\','?','@','\\','[','\\','\\','\\',']','\\','^','_','`','\\','{','\\','|','\\','}','\\','~']
  temp = {}
  for key,val in word_gram.items():
    if(key not in noise):
      temp[key] = val
  return temp

In [67]:
import string
import re
def get_string_punctuations():
  ans = re.escape(string.punctuation)
  return ans
punctuations = list(get_string_punctuations())
# punctuations

## 3(a) Part 1: Unigram

In [68]:
char_ug = {}
for key, val in word_ug.items():
    word = process_diacritic(key)
    for ch in word:
        if ch in char_ug.keys():
            char_ug[ch] += val
        else:
            char_ug[ch] = val
char_ug = remove_noise2(char_ug)
top100_3a1 = get_top_n(char_ug,100)
top100_3a1

,word,frequency
0,अ,1448939425
1,ा,584129700
2,क,466284203
3,र,444085371
4,े,427540427
...,...,...
95,►,27183
96,é,26676
97,к,26026
98,ل,25953


## 3(a) Part 2: Bigram

In [69]:
char_bg = {}
for key, val in word_ug.items():
    word = process_diacritic(key)
    for i in range(1, len(word)):
        ch = word[i-1] + word[i]
        if ch in char_bg.keys():
            char_bg[ch] += val
        else:
            char_bg[ch] = val
# char_bg
top100_3a2 = get_top_n(char_bg,100)
top100_3a2

,word,frequency
0,रअ,244674522
1,अर,164099359
2,कअ,125321919
3,सअ,107635225
4,नअ,105557724
...,...,...
95,सी,14664513
96,ली,14157365
97,िन,14140118
98,ाज,13861350


## 3(a) Part 3: Trigram

In [70]:
char_tg = {}
for key, val in word_ug.items():
    word = process_diacritic(key)
    for i in range(2, len(word)):
        ch = word[i-2] + word[i-1] + word[i]
        if ch in char_tg.keys():
            char_tg[ch] += val
        else:
            char_tg[ch] = val
top100_3a3 = get_top_n(char_tg,100)
top100_3a3

,word,frequency
0,अरअ,102213578
1,में,54736192
2,कअर,45165383
3,ारअ,37800776
4,अहअ,31704441
...,...,...
95,राज,6490142
96,कअत,6478159
97,अरि,6348186
98,िला,6270783


## 3(a) Part 4: Quadrigram

In [71]:
char_qg = {}
for key, val in word_ug.items():
    word = process_diacritic(key)
    for i in range(3, len(word)):
        ch = word[i-3] + word[i-2] + word[i-1] + word[i]
        if ch in char_qg.keys():
            char_qg[ch] += val
        else:
            char_qg[ch] = val
# char_qg
top100_3a4 = get_top_n(char_qg,100)
top100_3a4

,word,frequency
0,कअरअ,34080044
1,पअरअ,20926856
2,अहीं,11729258
3,नअही,10431119
4,अरअन,9628875
...,...,...
95,रअती,2762987
96,लिसअ,2760943
97,ारयअ,2758888
98,बातअ,2752972


## 3(c) Syllables

In [72]:
def process_akshar(word):
    word = process_diacritic(word)
    swar = ['अ','आ','इ','ई','उ','ऊ','ऋ','ए','ऐ','ओ','औ','अं','अः','ऻ','ि','ी','ू','ृ','ॄ','ॅ','ॆ','े', 'ं', 'ै','ॉ','ॊ','ो','ौ','्','ॎ','ॏ','ु']
    temp1 = []
    temp2 = []
    for ch in word:
        if ch in swar:
            temp2.append(ch)
            temp1.append(''.join(temp2))
            temp2 = []
        else:
            temp2.append(ch)
    return temp1

## 3(c) Part 1: Unigram

In [ ]:
syllab_ug = {}
for key, val in word_ug.items():
    syllables = process_akshar(key)
    for i in range(len(syllables)):
        syll_combo = syllables[i]
        if syll_combo in syllab_ug.keys():
            syllab_ug[syll_combo] += val
        else:
            syllab_ug[syll_combo] = val
# syllab_ug
top100_3c1 = get_top_n(syllab_ug,100)
top100_3c1

## 3(c) Part 2: Bigram

In [ ]:
syllab_bg = {}
for key, val in word_ug.items():
    syllables = process_akshar(key)
    for i in range(1, len(syllables)):
        syll_combo = syllables[i-1] + syllables[i]
        if syll_combo in syllab_bg.keys():
            syllab_bg[syll_combo] += val
        else:
            syllab_bg[syll_combo] = val
# syllab_bg
top100_3c2 = get_top_n(syllab_bg,100)
top100_3c2

## 3(c) Part 3: Trigram

In [ ]:
syllab_tg = {}
for key, val in word_ug.items():
    syllables = process_akshar(key)
    for i in range(2, len(syllables)):
        syll_combo = syllables[i-2] + syllables[i-1] + syllables[i]
        if syll_combo in syllab_tg.keys():
            syllab_tg[syll_combo] += val
        else:
            syllab_tg[syll_combo] = val
top100_3c3 = get_top_n(syllab_tg,100)
top100_3c3

## 3(d) Finding Zipfian Distribution

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
def draw_graph(df, nameofgraph):
  plt.plot(np.array(df.frequency))
  plt.ylabel('Frequency')
  plt.xlabel('Word_rank_as_per_frequency')
  plt.title(nameofgraph)
  return 

### Testing if Characters follow ZIPFIAN Distribution



Unigram

In [ ]:
draw_graph(top100_3a1,'Character Unigram')
print("Characters Unigram follows Zipfian's Distribution as evident from the following graph.")

Bigram

In [ ]:
draw_graph(top100_3a2,'Character Bigram')
print("Characters Bigram follows Zipfian's Distribution as evident from the following graph.")

Trigram

In [ ]:
draw_graph(top100_3a3,'Character Trigram')
print("Characters Trigram follows Zipfian's Distribution as evident from the following graph.")

Quadrigram

In [ ]:
draw_graph(top100_3a4,'Character Quadrigram')
print("Characters Quadrigram follows Zipfian's Distribution as evident from the following graph.")

### Testing if Words follow ZIPFIAN Distribution



Unigram

In [ ]:
draw_graph(top100_3b1,'Words Unigram')
print("Words Unigram follows Zipfian's Distribution as evident from the following graph.")

Bigram

In [ ]:
draw_graph(top100_3b1,'Words Bigram')
print("Words Bigram follows Zipfian's Distribution as evident from the following graph.")

Trigram

In [ ]:
draw_graph(top100_3b1,'Words Trigram')
print("Words Trigram follows Zipfian's Distribution as evident from the following graph.")

### Testing if Syllables have ZIPFIAN Distribution



Unigram

In [ ]:
draw_graph(top100_3c1,'Syllables Unigram')
print("Syllables Unigram follows Zipfian's Distribution as evident from the following graph.")

Bigram

In [ ]:
draw_graph(top100_3c2,'Syllables Bigram')
print("Syllables Bigram follows Zipfian's Distribution as evident from the following graph.")

Trigram

In [ ]:
draw_graph(top100_3c3,'Syllables Trigram')
print("Syllables Trigram follows Zipfian's Distribution as evident from the following graph.")

In [ ]:
# top100_3a1.to_csv('drive/My Drive/q3_outputs/top100_3a1.csv')
# top100_3a1.to_csv('drive/My Drive/q3_outputs/top100_3a2.csv')
# top100_3a1.to_csv('drive/My Drive/q3_outputs/top100_3a3.csv')
# top100_3a1.to_csv('drive/My Drive/q3_outputs/top100_3a4.csv')
# top100_3a1.to_csv('drive/My Drive/q3_outputs/top100_3b1.csv')
# top100_3a1.to_csv('drive/My Drive/q3_outputs/top100_3b2.csv')
# top100_3a1.to_csv('drive/My Drive/q3_outputs/top100_3b3.csv')
# top100_3a1.to_csv('drive/My Drive/q3_outputs/top100_3c1.csv')
# top100_3a1.to_csv('drive/My Drive/q3_outputs/top100_3c2.csv')
# top100_3a1.to_csv('drive/My Drive/q3_outputs/top100_3c3.csv')
